In [1]:
import pandas as pd
from functions import assembleDf, epochG, epochsG
from datetime import datetime, timedelta
import numpy as np
import sys
sys.path.append('..')
from pyglicko2.glicko2_tests import exampleCase
from pyglicko2.glicko2 import Player
import glicko2
import time


In [2]:
matches = pd.read_csv('../Data/matches_glicko2.csv',parse_dates = ['tourney_date'], 
                      infer_datetime_format = True)

Because memory constraints limit the batch size in epochsG, I will need to process incrementally.

In [3]:
eg_batches = [(s,f) for s,f in zip(range(0, 876_978-50_000,50_000),
                      range(50_000, 876978,50_000))] + [(850_000,876979)]

ratings_histories = []
players_dict = {}


In [4]:
eg_batches[0:1]

[(0, 50000)]

In [5]:
%%time
epochsG(matches[0:50_000],{},365)

CPU times: user 11.3 s, sys: 57.1 ms, total: 11.4 s
Wall time: 11.3 s


({113953: <pyglicko2.glicko2.Player at 0x7f7dd421cd00>,
  113954: <pyglicko2.glicko2.Player at 0x7f7dd421cd60>,
  113955: <pyglicko2.glicko2.Player at 0x7f7dd421cdc0>,
  113956: <pyglicko2.glicko2.Player at 0x7f7dd421ce20>,
  113957: <pyglicko2.glicko2.Player at 0x7f7dd421ce80>,
  113958: <pyglicko2.glicko2.Player at 0x7f7dd421cee0>,
  113959: <pyglicko2.glicko2.Player at 0x7f7dd421cf40>,
  113960: <pyglicko2.glicko2.Player at 0x7f7dd421cfa0>,
  113961: <pyglicko2.glicko2.Player at 0x7f7dc1d94040>,
  113962: <pyglicko2.glicko2.Player at 0x7f7dc1d940a0>,
  113963: <pyglicko2.glicko2.Player at 0x7f7dc1d94100>,
  113964: <pyglicko2.glicko2.Player at 0x7f7dc1d94160>,
  113965: <pyglicko2.glicko2.Player at 0x7f7dc1d941c0>,
  113966: <pyglicko2.glicko2.Player at 0x7f7dc1d94220>,
  113967: <pyglicko2.glicko2.Player at 0x7f7dc1d94280>,
  113968: <pyglicko2.glicko2.Player at 0x7f7dc1d942e0>,
  113969: <pyglicko2.glicko2.Player at 0x7f7dc1d94340>,
  113970: <pyglicko2.glicko2.Player at 0x7f7dc1d

In [6]:
%%time
for batch in eg_batches[0:1]:
    players_dict,rh = epochsG(matches[batch[0]:batch[1]],players_dict,365)
    ratings_histories += [rh]

CPU times: user 11.4 s, sys: 63 ms, total: 11.4 s
Wall time: 11.4 s


In [7]:
%%time

for batch in eg_batches[1:2]:
    players_dict,rh = epochsG(matches[batch[0]:batch[1]],players_dict,365)
    ratings_histories += [rh]

CPU times: user 4.29 s, sys: 38 ms, total: 4.32 s
Wall time: 4.3 s


In [8]:
%%time

for batch in eg_batches[2:3]:
    players_dict,rh = epochsG(matches[batch[0]:batch[1]],players_dict,365)
    ratings_histories += [rh]

CPU times: user 4.39 s, sys: 36.4 ms, total: 4.42 s
Wall time: 4.4 s


In [9]:
eg_batches

[(0, 50000),
 (50000, 100000),
 (100000, 150000),
 (150000, 200000),
 (200000, 250000),
 (250000, 300000),
 (300000, 350000),
 (350000, 400000),
 (400000, 450000),
 (450000, 500000),
 (500000, 550000),
 (550000, 600000),
 (600000, 650000),
 (650000, 700000),
 (700000, 750000),
 (750000, 800000),
 (800000, 850000),
 (850000, 876979)]

In [10]:
eg_batches[2:3]

[(100000, 150000)]

In [11]:
eg_batches_3_25k = [(s,f) for s,f in zip(range(150_000, 876_978-25_000,25_000),
                      range(175_000, 876978,25_000))] + [(875_000,876979)]

In [12]:
eg_batches_3_25k

[(150000, 175000),
 (175000, 200000),
 (200000, 225000),
 (225000, 250000),
 (250000, 275000),
 (275000, 300000),
 (300000, 325000),
 (325000, 350000),
 (350000, 375000),
 (375000, 400000),
 (400000, 425000),
 (425000, 450000),
 (450000, 475000),
 (475000, 500000),
 (500000, 525000),
 (525000, 550000),
 (550000, 575000),
 (575000, 600000),
 (600000, 625000),
 (625000, 650000),
 (650000, 675000),
 (675000, 700000),
 (700000, 725000),
 (725000, 750000),
 (750000, 775000),
 (775000, 800000),
 (800000, 825000),
 (825000, 850000),
 (850000, 875000),
 (875000, 876979)]

In [13]:
%%time

for batch in eg_batches_3_25k[0:1]:
    players_dict,rh = epochsG(matches[batch[0]:batch[1]],players_dict,365)
    ratings_histories += [rh]

CPU times: user 2.07 s, sys: 9.66 ms, total: 2.08 s
Wall time: 2.08 s


In [14]:
%%time

for batch in eg_batches_3_25k[1:2]:
    players_dict,rh = epochsG(matches[batch[0]:batch[1]],players_dict,365)
    ratings_histories += [rh]

CPU times: user 2.04 s, sys: 6.67 ms, total: 2.05 s
Wall time: 2.05 s


In [ ]:
batch

In [22]:
def splitBatch(atuple, divisions):
    batch_size = (atuple[0]-atuple[1])//divisions
    return [b for b in zip(range(atuple[0],atuple[1]-batch_size,batch_size),
         range(atuple[0]+batch_size,atuple[1],batch_size))]

In [25]:
splitBatch(eg_batches_3_25k[2],3)

[]

In [15]:
%%time

batch = eg_batches_3_25k[2]

players_dict,rh = epochsG(matches[batch[0]:batch[1]],players_dict,365)
ratings_histories += [rh]

KeyboardInterrupt: 

In [17]:
batch

(200000, 225000)

In [16]:
# eg_batches[4:5] this interval [(200000, 250000)] doesn't compute so I'll try to break it up